### Model to Streamlit

In [1]:
import sys
sys.path.insert(0, '../')


In [2]:
import file_paths
from basic_pitch_module.basic_pitch.inference import predict
from data_processing.midi2CP import CP
import pickle
import miditoolkit
import io


In [3]:
midi_data = predict('../temp/test/1.mp3')[1]

midi_stream = io.BytesIO()
midi_data.write(midi_stream)
midi_stream.seek(0)
midi_data = miditoolkit.midi.parser.MidiFile(file = midi_stream)


Predicting MIDI for ../temp/test/1.mp3...


In [4]:
midi_data


ticks per beat: 220
max tick: 198698
tempo changes: 1
time sig: 1
key sig: 0
markers: 0
lyrics: False
instruments: 1

In [5]:
#ith open(file_paths.cp_dict, 'rb') as f:
with open('/Users/d.haverkort/code/HugoA45/music_project/music_project/resources/CP.pkl', 'rb') as f:
    e2w, w2e = pickle.load(f)

    compact_classes = ['Bar', 'Position', 'Pitch', 'Duration']
    pad_CP = [e2w[subclass][f"{subclass} <PAD>"] for subclass in compact_classes]

    # preprocess input file
    CP_model = CP('/Users/d.haverkort/code/HugoA45/music_project/music_project/resources/CP.pkl')

    events, tokens = CP_model.prepare_data(midi_data, 512)


In [6]:
tokens.shape


(11, 512, 4)

In [7]:
from bert.load_midi_bert import load_model


In [8]:
dict_path = '/Users/d.haverkort/code/HugoA45/music_project/music_project/resources/CP.pkl'
ckpt_path = '/Users/d.haverkort/code/HugoA45/music_project/music_project/resources/pretrain_model.ckpt'
midi_bert = load_model(dict_path, ckpt_path)


Loading Dictionary


In [10]:
from predict.model_prediction import prediction

unts = prediction(tokens, midi_bert, task='composer_classification')


In [14]:
unts.value_counts()/len(unts)


Bethel Music (Religious)      0.636364
Yiruma (pop)                  0.181818
R. Sakamoto (Contemporary)    0.090909
H. Joe (Contemporary)         0.090909
dtype: float64